In [ ]:
import pandas as pd
from util import read_data, save, plt_encoding_error, error

In [ ]:
train, test_A, _ = read_data()

In [ ]:
pd.set_option('max_columns',100)

In [ ]:
rename = {'trueage': '年龄', 'alb':'白蛋白', 'glu':'血糖', 'crea':'肌酐', 'cho':'总胆固醇','tg':'甘油三酯','ua':'尿酸','hdlc':'高密度脂蛋白胆固醇','wbc':'白细胞计数','rbc':'红细胞计数','hct':'红细胞压积','mcv':'红细胞平均体积','mch':'红细胞平均血红蛋白量','mchc':'红细胞平均血红蛋白浓度','plt':'血小板计数','mpv':'血小板平均体积','pdw':'血小板体积分布宽度','lymph_a':'淋巴细胞%'}

In [ ]:
p_2014 = pd.read_csv('D:/contest/diabetes/dataverse_files/biomarker_dataset_CLHLS_2014-1.tab',sep='\s+')
p_2012 = pd.read_csv('D:/contest/diabetes/dataverse_files/biomarker_dataset_CLHLS_2012-1.tab',sep='\s+')
p_2008 = pd.read_csv('D:/contest/diabetes/dataverse_files/biomarker_dataset_CLHLS_2008-1.tab',sep='\s+')

In [ ]:
p_2014 = p_2014.rename(columns=rename)
p_2012 = p_2012.rename(columns=rename)
p_2008 = p_2008.rename(columns=rename)

In [ ]:
p_total = pd.concat([p_2008, p_2012, p_2014])

In [ ]:
p_col = p_total.columns.tolist()
original_col = train.columns.tolist()

In [ ]:
contain_list = []
for i in p_col:
    if i in original_col:
        contain_list.append(i)

In [ ]:
drop_list = list(set(p_col) - set(original_col))
p_total.drop(drop_list, axis=1, inplace=True)

In [ ]:
p_total.dropna(subset = ['血糖'], axis=0, inplace=True)

In [ ]:
# 异常值处理
p_total = p_total[p_total['血糖']<50]
p_total = p_total[p_total['年龄']<95]
p_total = p_total[p_total['血小板计数']<800]
p_total = p_total[p_total['甘油三酯']<20]
p_total = p_total[p_total['肌酐']<300]
p_total = p_total[p_total['红细胞计数']<20]

In [ ]:
# 粗略调整年龄分布
large_age_70 = p_total[(p_total['年龄']>70) & (p_total['年龄']<80)].sample(frac=0.8, replace=False, random_state=1, axis=0)
large_age_80 = p_total[p_total['年龄']>80].sample(frac=0.3, replace=False, random_state=1, axis=0)
p_total = p_total[p_total['年龄']<70]
p_total = pd.concat([p_total, large_age_70, large_age_80])

In [ ]:
p_total.describe()

In [ ]:
train.describe()

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns #要注意的是一旦导入了seaborn，matplotlib的默认作图风格就会被覆盖成seaborn的格式
# #train = train.fillna(-1)
# sns.distplot(train['年龄'])
# plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns #要注意的是一旦导入了seaborn，matplotlib的默认作图风格就会被覆盖成seaborn的格式
# #train = train.fillna(-1)
# sns.distplot(p_total['年龄'])
# plt.show()

In [ ]:
result = pd.concat([p_total,train])

In [ ]:
result.head()

In [ ]:
result['性别'] = result['性别'].map({'男':1, '女':0, '？？':1})
result = result.rename(columns={'性别':'gender','年龄':'age','体检日期':'date'})

In [ ]:
result.to_csv("data_m.csv",index=False, encoding="utf-8")